In [2]:
import numpy as np

In [3]:
print("Hello World!")

Hello World!


In [4]:
print("Apparently Google Collab cannot load an empty notebook")

Apparently Google Collab cannot load an empty notebook


In [5]:
ones = np.ones(100)

In [7]:
ones_squared = ones**2

In [8]:
np.sum(ones-ones_squared)

0.0

In [9]:
print("Who would have thunk?")

Who would have thunk?
